In [4]:
import os
import sys
# allow loading modules from local directory.
sys.path.insert(1, '/home/jovyan/work/code')

from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import SERVICE_NAME, Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.instrumentation.langchain import LangchainInstrumentor

# Logging (Experimental)
from opentelemetry._logs import set_logger_provider
from opentelemetry.exporter.otlp.proto.grpc._log_exporter import (
    OTLPLogExporter,
)
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler
from opentelemetry.sdk._logs.export import BatchLogRecordProcessor
from opentelemetry.sdk.resources import Resource

#from otel_grpc import configure_otel_otlp

from config import VectorDBConfig, EmbeddingConfig, ProcessingConfig, ChatConfig
from pipeline import DocumentPipeline
from localrag import LocalRAG

import nest_asyncio
import asyncio 
import logging

# Enable nested event loops
nest_asyncio.apply()

# Initialise and Setup OpenTelemetry for the session
resource = Resource(attributes={
  SERVICE_NAME:  os.getenv('OTEL_SERVICE_NAME', 'jupyter-demo')
})
provider = TracerProvider(resource=resource)
processor = BatchSpanProcessor(OTLPSpanExporter(endpoint=os.getenv("OTEL_EXPORTER_OTLP_ENDPOINT")))
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)

 # Configure Logging
#configure_otel_otlp( os.getenv('OTEL_SERVICE_NAME', 'jupyter-demo'), endpoint=os.getenv("OTEL_EXPORTER_OTLP_ENDPOINT"))

logger_provider = LoggerProvider(
    resource=resource
)
set_logger_provider(logger_provider)

exporter = OTLPLogExporter(insecure=True)
logger_provider.add_log_record_processor(BatchLogRecordProcessor(exporter))
handler = LoggingHandler(level=logging.NOTSET, logger_provider=logger_provider)

# Attach OTLP handler to root logger
logging.getLogger().addHandler(handler)

# logging.basicConfig()
#logging.basicConfig(format = "%(asctime)s:%(levelname)s:%(message)s", level = logging.DEBUG)
logging.root.setLevel(logging.INFO)


LangchainInstrumentor().instrument()
logger = logging.getLogger(__name__)
tracer = trace.get_tracer(__name__)


ERROR:root:This is a log message


In [ ]:
# Get the connection strings and configuration passed from Aspire AppHost

def parse_ollama_connection(conn_str):
    parts = conn_str.split(';')
    endpoint = next(p.split('=')[1] for p in parts if p.startswith('Endpoint='))
    model = next(p.split('=')[1] for p in parts if p.startswith('Model='))
    return endpoint, model

chat_conn = os.getenv('ConnectionStrings__chat-model', 'Endpoint=http://ollama:11434;Model=phi3.5')
chat_model_url, chat_model_id = parse_ollama_connection(chat_conn)

embeddings_conn = os.getenv('ConnectionStrings__embedding-model', 'Endpoint=http://ollama:11434;Model=mxbai-embed-large')
embedding_model_url, embeddings_model = parse_ollama_connection(embeddings_conn)

qdrant_conn = os.getenv('ConnectionStrings__qdrant_http', 'Endpoint=http://qdrant:6334;Key=aMjJKx0t1a6E9hysaCacWz')
parts = qdrant_conn.split(';')
qdrant_url = next(p.split('=')[1] for p in parts if p.startswith('Endpoint='))
qdrant_key = next(p.split('=')[1] for p in parts if p.startswith('Key='))

vector_db_config = VectorDBConfig(
    url=qdrant_url,
    api_key=qdrant_key,
    collection_name="embedding-demo-aspire-inc-yml"
)
chat_config = ChatConfig(
    model_name=chat_model_id,
    base_url=chat_model_url
)
embedding_config = EmbeddingConfig(
    model_name=embeddings_model,
    base_url=embedding_model_url
)
processing_config = ProcessingConfig(
    chunk_size=500,
    chunk_overlap=50,
    add_metadata=True,
    extract_code_entities=True
)

print(vector_db_config)
print(embedding_config)
print(chat_config)

In [ ]:
# Ingest a GitHub repository and import into our vector store

# Initialize the pipeline
pipeline = DocumentPipeline(
    vector_db_config=vector_db_config,
    embedding_config=embedding_config, 
    processing_config=processing_config
)

###Process git repository
#pipeline.process_repository("https://github.com/dotnet/docs-aspire")

# Or process local directory
#pipeline.process_local_directory("./docs")


In [ ]:
# Test our RAG Solution

def demonstrate_local_rag(rag):
    """Demonstrate how to use the LocalRAG class."""    
    # Example questions to test
    questions = [
        "Why should I know about .Net Aspire?",
        "Is .Net Aspire an alternative to Kubernetes?"
    ]
    with tracer.start_as_current_span("Entering questions loop."):
        for question in questions:
            print(f"Question: {question}")
            print("\nRelevant chunks:")
            #with tracer.start_as_current_span("rag get chunks"):
            # chunks = rag.get_relevant_chunks(question, k=5)
            # for i, chunk in enumerate(chunks, 1):
            #     print(f"\nChunk {i}:")
            #     print(f"Source: {chunk.metadata.get('file_path', 'Unknown')}")
            #      print(f"Content: {chunk.page_content[:200]}...")
                
            print("\nGenerated Answer:")
            with tracer.start_as_current_span("Retrieve answers."):
                answer = rag.retrieve_and_answer(question, k=6)
                print(answer)
                print("\n" + "="*80 + "\n")

rag = LocalRAG(
    vector_db_config=vector_db_config,
    embedding_config=embedding_config, 
    chat_config=chat_config
)
with tracer.start_as_current_span("Starting demo"):
    demonstrate_local_rag(rag)